In [1]:
import sys
from pathlib import Path

# Set manually based on your repo's structure
repo_root = Path.cwd().parent  # or Path.cwd() if you're in the repo root
sys.path.append(str(repo_root))

# ArrayManager Example Notebook

In [2]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import numpy as np
import dask.array as da

In [3]:
# Step 1: Create a synthetic 5D image (T, C, Z, Y, X)
t, c, z, y, x = 5, 2, 64, 1024, 1024
np_data = np.random.randint(0, 5000, size=(t, c, z, y, x), dtype=np.uint16)
dask_data = da.from_array(np_data, chunks=(1, 1, 4, 64, 64))

In [6]:
# Step 2: Define metadata
metadata = {
            "size": [dask_data.shape],
            "scales": [(4.0, 0.65, 0.65)],
            "units": ("micrometer", "micrometer", "micrometer"),
            "time_increment": 1.0,
            "time_increment_unit": "s",
            "channel_names": ["Red", "Green"],
            "channel_colors": [16711680, 255],
            "dtype": "uint16",
            "axes": "tczyx",
        }

In [7]:
# Step 3: Instantiate ArrayManager
dataset = mm.ArrayManager(dask_data, metadata)
print("Loaded data shape:", dataset.data[0].shape)

Loaded data shape: (5, 2, 64, 1024, 1024)


In [8]:
# Step 4: Build pyramid (starting from base level)
dataset.build_pyramid(num_levels=3)
print("New pyramid levels:", [arr.shape for arr in dataset.data])

Requested start level 0
Resolution layer already available
New pyramid levels: [(5, 2, 64, 1024, 1024), (5, 2, 32, 512, 512), (5, 2, 16, 256, 256)]


In [9]:
# Step 5: Visualize
viewer = dataset.visualize()

In [11]:
# Reorder channels
dataset.reorder_channels([1,0])
# Confirm channels reodered
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

✅ Channels reordered to [1, 0]
SIZE: [(5, 2, 64, 1024, 1024), (5, 2, 32, 512, 512), (5, 2, 16, 256, 256)]
SCALES: [(4.0, 0.65, 0.65), (8.0, 1.3, 1.3), (16.0, 2.6, 2.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Green', 'Red']
CHANNEL_COLORS: [255, 16711680]
DTYPE: uint16
AXES: tczyx
PLANE_FILES: None


In [13]:
# Change metadata
dataset.update_metadata({
    "channel_names":["Blue", "Red"],
    "time_increment_unit": "m",
})
# Confirm metadata changed
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

✅ Updated metadata entry 'channel_names'
✅ Updated metadata entry 'time_increment_unit'
SIZE: [(5, 2, 64, 1024, 1024), (5, 2, 32, 512, 512), (5, 2, 16, 256, 256)]
SCALES: [(4.0, 0.65, 0.65), (8.0, 1.3, 1.3), (16.0, 2.6, 2.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: m
CHANNEL_NAMES: ['Blue', 'Red']
CHANNEL_COLORS: [255, 16711680]
DTYPE: uint16
AXES: tczyx
PLANE_FILES: None
